Deep Learning
=============

Assignment 3 and Solutions
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # Hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
IMAGE_SIZE = 28
NUM_LABELS = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, IMAGE_SIZE * IMAGE_SIZE)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(NUM_LABELS) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

In [5]:
# Logistic regression with L2 regulariation
NUM_LABELS = 10
IMAGE_SIZE = 28
batch_size = 128
num_steps = 801
beta = 0.001

In [6]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, IMAGE_SIZE * IMAGE_SIZE))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_output = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, NUM_LABELS]))
    biases_output = tf.Variable(tf.zeros([NUM_LABELS]))
  
    # Training computation.
    score = tf.matmul(tf_train_dataset, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))\
        + beta * tf.nn.l2_loss(weights_output)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    score_valid = tf.matmul(tf_valid_dataset, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(score_valid)
    score_test = tf.matmul(tf_test_dataset, weights_output) + biases_output
    test_prediction = tf.nn.softmax(score_test)

In [7]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.335505
Minibatch accuracy: 9.4%
Validation accuracy: 14.8%
Minibatch loss at step 100: 4.637360
Minibatch accuracy: 75.0%
Validation accuracy: 71.4%
Minibatch loss at step 200: 4.142784
Minibatch accuracy: 78.9%
Validation accuracy: 73.7%
Minibatch loss at step 300: 3.687744
Minibatch accuracy: 69.5%
Validation accuracy: 74.6%
Minibatch loss at step 400: 3.367444
Minibatch accuracy: 73.4%
Validation accuracy: 75.6%
Minibatch loss at step 500: 2.411777
Minibatch accuracy: 79.7%
Validation accuracy: 75.9%
Minibatch loss at step 600: 2.350111
Minibatch accuracy: 82.0%
Validation accuracy: 75.9%
Minibatch loss at step 700: 2.668043
Minibatch accuracy: 70.3%
Validation accuracy: 76.9%
Minibatch loss at step 800: 1.814509
Minibatch accuracy: 83.6%
Validation accuracy: 77.2%
Test accuracy: 85.1%


In [8]:
# Simple neural network with L2 regulariation
NUM_LABELS = 10
IMAGE_SIZE = 28
batch_size = 128
num_hidden1 = 1024
num_steps = 10001
beta = 0.001

In [9]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, IMAGE_SIZE * IMAGE_SIZE))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, num_hidden1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden1]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden1, NUM_LABELS]))
    biases_output = tf.Variable(tf.zeros([NUM_LABELS]))
  
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden)
    
    score = tf.matmul(hidden1, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))\
        + beta * (tf.nn.l2_loss(weights_hidden1) + tf.nn.l2_loss(weights_output))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    score_hidden_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    valid_prediction = tf.nn.softmax(score_valid)
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    test_prediction = tf.nn.softmax(score_test)

In [10]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 655.929932
Minibatch accuracy: 9.4%
Validation accuracy: 29.1%
Minibatch loss at step 500: 199.772766
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 116.680862
Minibatch accuracy: 73.4%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 68.372192
Minibatch accuracy: 89.1%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 41.298187
Minibatch accuracy: 89.8%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 25.203154
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.490561
Minibatch accuracy: 85.9%
Validation accuracy: 86.5%
Minibatch loss at step 3500: 9.642866
Minibatch accuracy: 88.3%
Validation accuracy: 87.1%
Minibatch loss at step 4000: 5.964278
Minibatch accuracy: 90.6%
Validation accuracy: 87.6%
Minibatch loss at step 4500: 3.817868
Minibatch accuracy: 89.1%
Validation accuracy: 87.4%
Minibatch loss at step 5000: 2.522035
Minibatch accuracy: 90.6%
Validatio

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [11]:
# Logistic regression with L2 regulariation under overfitting
# by restricting training data to just 3 batches.
NUM_LABELS = 10
IMAGE_SIZE = 28
batch_size = 128
num_steps = 801
beta = 0.001

In [12]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, IMAGE_SIZE * IMAGE_SIZE))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_output = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, NUM_LABELS]))
    biases_output = tf.Variable(tf.zeros([NUM_LABELS]))
  
    # Training computation.
    score = tf.matmul(tf_train_dataset, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))\
        + beta * tf.nn.l2_loss(weights_output)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    score_valid = tf.matmul(tf_valid_dataset, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(score_valid)
    score_test = tf.matmul(tf_test_dataset, weights_output) + biases_output
    test_prediction = tf.nn.softmax(score_test)

In [13]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = (step % 3) * batch_size
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 23.278521
Minibatch accuracy: 6.2%
Validation accuracy: 9.9%
Minibatch loss at step 100: 3.226169
Minibatch accuracy: 87.5%
Validation accuracy: 63.4%
Minibatch loss at step 200: 2.423492
Minibatch accuracy: 100.0%
Validation accuracy: 65.0%
Minibatch loss at step 300: 2.205252
Minibatch accuracy: 99.2%
Validation accuracy: 66.2%
Minibatch loss at step 400: 1.977893
Minibatch accuracy: 100.0%
Validation accuracy: 67.0%
Minibatch loss at step 500: 1.781301
Minibatch accuracy: 100.0%
Validation accuracy: 67.6%
Minibatch loss at step 600: 1.629060
Minibatch accuracy: 100.0%
Validation accuracy: 68.2%
Minibatch loss at step 700: 1.472420
Minibatch accuracy: 100.0%
Validation accuracy: 68.7%
Minibatch loss at step 800: 1.329631
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Test accuracy: 76.2%


In [14]:
# Simple neural network with L2 regulariation under overfitting
# by restricting training data to just 3 batches.
NUM_LABELS = 10
IMAGE_SIZE = 28
batch_size = 128
num_hidden1 = 1024
num_steps = 10001
beta = 0.001

In [15]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, IMAGE_SIZE * IMAGE_SIZE))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, num_hidden1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden1]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden1, NUM_LABELS]))
    biases_output = tf.Variable(tf.zeros([NUM_LABELS]))
  
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden)
    
    score = tf.matmul(hidden1, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))\
        + beta * (tf.nn.l2_loss(weights_hidden1) + tf.nn.l2_loss(weights_output))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    score_hidden_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    valid_prediction = tf.nn.softmax(score_valid)
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    test_prediction = tf.nn.softmax(score_test)

In [16]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = (step % 3) * batch_size
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 632.554932
Minibatch accuracy: 7.0%
Validation accuracy: 30.1%
Minibatch loss at step 500: 190.373688
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 115.453049
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 70.017151
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 2000: 42.462204
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 2500: 25.751610
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 3000: 15.617757
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 3500: 9.473000
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 4000: 5.749737
Minibatch accuracy: 100.0%
Validation accuracy: 76.1%
Minibatch loss at step 4500: 3.497044
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 5000: 2.133898
Minibatch accuracy: 100.0%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [17]:
# Simple neural network with dropout
NUM_LABELS = 10
IMAGE_SIZE = 28
batch_size = 128
num_hidden1 = 1024
num_steps = 10001
beta = 0.001
keep_prob = 0.3

In [18]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, IMAGE_SIZE * IMAGE_SIZE))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, num_hidden1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden1]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden1, NUM_LABELS]))
    biases_output = tf.Variable(tf.zeros([NUM_LABELS]))
  
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden)
    # Add dropout
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
    
    score = tf.matmul(hidden1_drop, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    score_hidden_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    valid_prediction = tf.nn.softmax(score_valid)
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    test_prediction = tf.nn.softmax(score_test)

In [19]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 635.219971
Minibatch accuracy: 7.8%
Validation accuracy: 29.5%
Minibatch loss at step 500: 79.608208
Minibatch accuracy: 71.9%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 44.552734
Minibatch accuracy: 64.8%
Validation accuracy: 76.0%
Minibatch loss at step 1500: 31.595276
Minibatch accuracy: 60.9%
Validation accuracy: 72.6%
Minibatch loss at step 2000: 13.828009
Minibatch accuracy: 60.2%
Validation accuracy: 72.0%
Minibatch loss at step 2500: 4.727857
Minibatch accuracy: 61.7%
Validation accuracy: 73.5%
Minibatch loss at step 3000: 6.518710
Minibatch accuracy: 52.3%
Validation accuracy: 73.5%
Minibatch loss at step 3500: 19.362532
Minibatch accuracy: 58.6%
Validation accuracy: 72.9%
Minibatch loss at step 4000: 15.043457
Minibatch accuracy: 64.8%
Validation accuracy: 73.8%
Minibatch loss at step 4500: 10.220227
Minibatch accuracy: 64.8%
Validation accuracy: 74.1%
Minibatch loss at step 5000: 6.970784
Minibatch accuracy: 60.9%
Validation

In [20]:
# Simple neural network with dropout under overfitting
# by restricting training data to just 3 batches.
NUM_LABELS = 10
IMAGE_SIZE = 28
batch_size = 128
num_hidden1 = 1024
num_steps = 10001
beta = 0.001

In [21]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, IMAGE_SIZE * IMAGE_SIZE))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, num_hidden1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden1]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden1, NUM_LABELS]))
    biases_output = tf.Variable(tf.zeros([NUM_LABELS]))
  
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden)
    # Add dropout
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
    
    score = tf.matmul(hidden1_drop, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    score_hidden_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    valid_prediction = tf.nn.softmax(score_valid)
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    test_prediction = tf.nn.softmax(score_test)

In [22]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = (step % 3) * batch_size
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 602.876953
Minibatch accuracy: 10.9%
Validation accuracy: 34.4%
Minibatch loss at step 500: 0.176566
Minibatch accuracy: 99.2%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 1.866451
Minibatch accuracy: 97.7%
Validation accuracy: 77.9%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 0.083202
Minibatch accuracy: 99.2%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 0.153027
Minibatch accuracy: 99.2%
Validation accuracy: 77.6%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Validation

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [7]:
# Deep learning with 4-layers neural networks (3 hidden layers + 1 output layer), 
# dropout, L2 regularization and learning rate decay.
NUM_LABELS = 10
IMAGE_SIZE = 28
batch_size = 128
num_hidden1 = 1024
num_hidden2 = 600
num_hidden3 = 200
sd_hidden1 = np.sqrt(2.0/1000)
sd_hidden2 = np.sqrt(2.0/num_hidden1)
sd_hidden3 = np.sqrt(2.0/num_hidden2)
sd_output = np.sqrt(2.0/num_hidden3)
keep_prob1 = 0.9
keep_prob2 = 0.8
keep_prob3 = 0.7
beta1 = 0.0001
beta2 = 0.0001
beta3 = 0.0001
beta4 = 0.0001
num_steps = 20001
decay_steps = 1000
decay_rate = 0.96

In [23]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, IMAGE_SIZE * IMAGE_SIZE))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, num_hidden1], stddev=sd_hidden1))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden1]))
    weights_hidden2 = tf.Variable(
        tf.truncated_normal([num_hidden1, num_hidden2], stddev=sd_hidden2))
    biases_hidden2 = tf.Variable(tf.zeros([num_hidden2]))
    weights_hidden3 = tf.Variable(
        tf.truncated_normal([num_hidden2, num_hidden3], stddev=sd_hidden3))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden3]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden3, NUM_LABELS]))
    biases_output = tf.Variable(tf.zeros([NUM_LABELS]))
  
    # Computate training.
    score_hidden1 = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden1)
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob1)
    score_hidden2 = tf.matmul(hidden1_drop, weights_hidden2) + biases_hidden2
    hidden2 = tf.nn.relu(score_hidden2)
    hidden2_drop = tf.nn.dropout(hidden2, keep_prob2)
    score_hidden3 = tf.matmul(hidden2_drop, weights_hidden3) + biases_hidden3
    hidden3 = tf.nn.relu(score_hidden3)
    hidden3_drop = tf.nn.dropout(hidden3, keep_prob3)
    
    score = tf.matmul(hidden3_drop, weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))\
        + beta1 * tf.nn.l2_loss(weights_hidden1)\
        + beta2 * tf.nn.l2_loss(weights_hidden2)\
        + beta3 * tf.nn.l2_loss(weights_hidden3)\
        + beta4 * tf.nn.l2_loss(weights_output))
    
    # TODO: Add learning rate decay
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(
        starter_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
  
    # Optimizer.
    optimizer = (tf.train.GradientDescentOptimizer(learning_rate)
                   .minimize(loss, global_step=global_step))
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    
    # TODO (Bowen): Compute validation.
    score_hidden1 = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden1)
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob1)
    score_hidden2 = tf.matmul(hidden1_drop, weights_hidden2) + biases_hidden2
    hidden2 = tf.nn.relu(score_hidden2)
    hidden2_drop = tf.nn.dropout(hidden2, keep_prob2)
    score_hidden3 = tf.matmul(hidden2_drop, weights_hidden3) + biases_hidden3
    hidden3 = tf.nn.relu(score_hidden3)
    hidden3_drop = tf.nn.dropout(hidden3, keep_prob3)
    
    score_hidden1_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    valid_prediction = tf.nn.softmax(score_valid)
    
    # TODO (Bowen): Compute testing.
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    test_prediction = tf.nn.softmax(score_test)